
Originally created by Chelsey McGowan-Yallop, SAMS-UHI (sa06cm@sams.ac.uk)

Modified by Muriel Dunn for AFKABAN

This script uses model-predicted TS(f) spectra to train a machine learning
classifier, performs nested cross-validation, applies the classifier to
measured TS(f) spectra and outputs results files.

To use a different classifier, see the list of supported classifiers at:
https://github.com/hyperopt/hyperopt-sklearn and set as clf.

Sometimes the initial hyperparameter configuration selected by HyperOpt in each
split in the outer loop will be unsuccessful and all trials will fail. The
retry decorator forces it to try again until retry_limit is reached.

OUTPUT FILES:
    _NestedCV.pkl contains results of nested cross-validation procedure
    _Predictions.pkl contains measured TS(f) spectra with predicted labels
    _BestParams.pkl contains the optimal hyperparameters for the model
"""

In [1]:
import time
import os.path
import numpy as np
import pandas as pd
import pickle
import scipy
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, confusion_matrix
#from sklearn.neighbors import KNeighborsClassifier
import hyperopt
from hyperopt import tpe
from hpsklearn import HyperoptEstimator, k_neighbors_classifier, svc, lightgbm_classification, gaussian_nb, any_preprocessing,min_max_scaler
import lightgbm
from datetime import timedelta, date
from tenacity import retry, stop_after_attempt
import AZKABANML
import sys, errno  


# Read the dataframes

In [26]:
# Set paths
path = 'F:/AFKABAN/'
classifypath = 'F:/AFKABAN/Classify/'

df_120 = pd.read_feather(f'{path}/df_120_balanced.feather')
df_200 = pd.read_feather(f'{path}/df_200_balanced.feather')

df_120_sigbs = pd.read_feather(f'{path}/df_120_sigbs_balanced.feather')
df_200_sigbs = pd.read_feather(f'{path}/df_200_sigbs_balanced.feather')

# Run different iterations


## Preprocessing
Any! Give better results (+~0.2 on mean F1 score), either scale or min,max scaler or PCA.

In [5]:
# -- USER-DETERMINED PARAMETERS -----------------------------------------------
path = 'F:/AFKABAN/'
classifypath = 'F:/AFKABAN/Classify/'
d1 = date.today().strftime("%d-%m-%Y")

# NESTED CROSS-VALIDATION
#preprocessing = [min_max_scaler(unique_id)] # List of sklearn pre-processing modules
ex_preprocessing = [] # As above, see help(HyperoptEstimator) for info
n_splits = 10 # Value of k for k-fold cross-validation in outer loop
n_folds = 10 # Value of k for k-fold cross-validation in inner loop
max_evals = 50 # No. of HyperOpt trials
timeout = 300 # HyperOpt trial timeout (seconds)
n_jobs = -1 # No. of jobs to run in parallel; -1 uses all processors
retry_limit = 3 # No. of times to retry before failing

### kNN

In [24]:
# CLASSIFIER kNN 120
unique_id = 'kNN_'+ d1 +'_120_balanced' # Unique ID for output file paths
clf = k_neighbors_classifier(unique_id)  # Classifier
AZKABANML.main_classify(df_120_balanced, clf, unique_id, classifypath, preprocessing=any_preprocessing(unique_id))

Beginning fold 1 of 10
100%|██████████████████████████████████████████████| 50/50 [00:04<00:00,  4.27s/trial, best loss: 0.009061833688699394]
Beginning fold 2 of 10
100%|██████████████████████████████████████████████| 17/17 [00:04<00:00,  4.16s/trial, best loss: 0.007995735607675858]


100%|██████████████████████████████████████████████| 50/50 [00:05<00:00,  5.58s/trial, best loss: 0.007995735607675858]
Beginning fold 3 of 10
100%|███████████████████████████████████████████████| 35/35 [00:04<00:00,  4.07s/trial, best loss: 0.01012793176972282]


100%|██████████████████████████████████████████████| 50/50 [00:05<00:00,  5.60s/trial, best loss: 0.009594882729211052]
Beginning fold 4 of 10
100%|██████████████████████████████████████████████| 50/50 [00:04<00:00,  4.19s/trial, best loss: 0.009061833688699394]
Beginning fold 5 of 10
100%|████████████████████████████████████████████████| 2/2 [00:04<00:00,  4.12s/trial, best loss: 0.049040511727078906]


100%|██████████████████████████████████████████████| 50/50 [00:06<00:00,  6.50s/trial, best loss: 0.009594882729211052]
Beginning fold 6 of 10
100%|██████████████████████████████████████████████| 20/20 [00:07<00:00,  7.65s/trial, best loss: 0.010655301012253648]


100%|██████████████████████████████████████████████| 50/50 [00:06<00:00,  6.18s/trial, best loss: 0.010122535961640922]
Beginning fold 7 of 10
100%|███████████████████████████████████████████████| 38/38 [00:04<00:00,  4.14s/trial, best loss: 0.01331912626531695]


100%|███████████████████████████████████████████████| 50/50 [00:04<00:00,  4.23s/trial, best loss: 0.01331912626531695]
Beginning fold 8 of 10
100%|██████████████████████████████████████████████| 50/50 [00:04<00:00,  4.03s/trial, best loss: 0.010122535961640922]
Beginning fold 9 of 10
100%|█████████████████████████████████████████████████| 5/5 [00:05<00:00,  5.30s/trial, best loss: 0.01331912626531695]


100%|██████████████████████████████████████████████| 50/50 [00:04<00:00,  4.48s/trial, best loss: 0.012786361214704334]
Beginning fold 10 of 10
100%|██████████████████████████████████████████████| 23/23 [00:04<00:00,  4.22s/trial, best loss: 0.015450186467767746]


100%|██████████████████████████████████████████████| 50/50 [00:06<00:00,  6.08s/trial, best loss: 0.014917421417155019]
Retraining model on full dataset
100%|██████████████████████████████████████████████| 41/41 [00:07<00:00,  7.95s/trial, best loss: 0.010071942446043147]


100%|██████████████████████████████████████████████| 50/50 [00:04<00:00,  4.50s/trial, best loss: 0.010071942446043147]


In [25]:
# CLASSIFIER kNN 120
unique_id = 'kNN_'+ d1 +'_200_balanced' # Unique ID for output file paths
clf = k_neighbors_classifier(unique_id)  # Classifier
AZKABANML.main_classify(df_200_balanced, clf, unique_id, classifypath, preprocessing=any_preprocessing(unique_id))

Beginning fold 1 of 10
100%|██████████████████████████████████████████████| 50/50 [00:06<00:00,  6.92s/trial, best loss: 0.012188659247482803]
Beginning fold 2 of 10
100%|██████████████████████████████████████████████| 17/17 [00:04<00:00,  4.25s/trial, best loss: 0.013248542660307394]


100%|██████████████████████████████████████████████| 50/50 [00:06<00:00,  6.06s/trial, best loss: 0.012188659247482803]
Beginning fold 3 of 10
100%|██████████████████████████████████████████████| 35/35 [00:05<00:00,  5.88s/trial, best loss: 0.011658717541070507]


100%|██████████████████████████████████████████████| 50/50 [00:04<00:00,  4.11s/trial, best loss: 0.011658717541070507]
Beginning fold 4 of 10
100%|███████████████████████████████████████████████| 50/50 [00:04<00:00,  4.17s/trial, best loss: 0.01642819289878117]
Beginning fold 5 of 10
100%|████████████████████████████████████████████████| 2/2 [00:04<00:00,  4.37s/trial, best loss: 0.018018018018018056]


100%|██████████████████████████████████████████████| 50/50 [00:06<00:00,  6.87s/trial, best loss: 0.011658717541070507]
Beginning fold 6 of 10
100%|██████████████████████████████████████████████| 20/20 [00:04<00:00,  4.17s/trial, best loss: 0.011128775834658211]


100%|██████████████████████████████████████████████| 50/50 [00:05<00:00,  5.99s/trial, best loss: 0.011128775834658211]
Beginning fold 7 of 10
100%|███████████████████████████████████████████████| 38/38 [00:05<00:00,  5.50s/trial, best loss: 0.01642819289878117]


100%|███████████████████████████████████████████████| 50/50 [00:05<00:00,  5.55s/trial, best loss: 0.01642819289878117]
Beginning fold 8 of 10
100%|██████████████████████████████████████████████| 50/50 [00:05<00:00,  5.06s/trial, best loss: 0.010063559322033844]
Beginning fold 9 of 10
100%|████████████████████████████████████████████████| 5/5 [00:04<00:00,  4.02s/trial, best loss: 0.014830508474576232]


100%|███████████████████████████████████████████████| 50/50 [00:05<00:00,  5.54s/trial, best loss: 0.01218220338983056]
Beginning fold 10 of 10
100%|████████████████████████████████████████████████| 23/23 [00:05<00:00,  5.61s/trial, best loss: 0.0201271186440678]


100%|███████████████████████████████████████████████| 50/50 [00:04<00:00,  4.08s/trial, best loss: 0.01959745762711862]
Retraining model on full dataset
100%|██████████████████████████████████████████████| 41/41 [00:04<00:00,  4.14s/trial, best loss: 0.011921793037672823]


100%|██████████████████████████████████████████████| 50/50 [00:04<00:00,  4.16s/trial, best loss: 0.011921793037672823]


#### lightGBM

In [31]:
# CLASSIFIER lightGBM 120
unique_id = 'lightGBM_'+ d1 +'_120_balanced' # Unique ID for output file paths
clf = lightgbm_classification(unique_id)  # Classifier
AZKABANML.main_classify(df_120, clf, unique_id, classifypath, preprocessing=any_preprocessing(unique_id))

Beginning fold 1 of 10
100%|██████████████████████████████████████████████| 50/50 [00:43<00:00, 43.98s/trial, best loss: 0.006396588486140775]
Beginning fold 2 of 10
100%|█████████████████████████████████████████████| 17/17 [01:45<00:00, 105.76s/trial, best loss: 0.011194029850746245]


100%|██████████████████████████████████████████████| 50/50 [00:28<00:00, 28.33s/trial, best loss: 0.006396588486140775]
Beginning fold 3 of 10
100%|██████████████████████████████████████████████| 35/35 [00:25<00:00, 25.84s/trial, best loss: 0.004264392324093813]


100%|██████████████████████████████████████████████| 50/50 [01:04<00:00, 64.25s/trial, best loss: 0.004264392324093813]
Beginning fold 4 of 10
100%|█████████████████████████████████████████████| 50/50 [01:48<00:00, 108.20s/trial, best loss: 0.007995735607675858]
Beginning fold 5 of 10
100%|█████████████████████████████████████████████████| 2/2 [00:20<00:00, 20.01s/trial, best loss: 0.03624733475479747]


100%|██████████████████████████████████████████████| 50/50 [00:28<00:00, 28.21s/trial, best loss: 0.007462686567164201]
Beginning fold 6 of 10
100%|███████████████████████████████████████████████████████████████████| 1/1 [05:05<00:00, 305.73s/trial, best loss=?]
Resuming cross-validation from fold 6
Beginning fold 6 of 10
100%|██████████████████████████████████████████████| 18/18 [00:22<00:00, 22.23s/trial, best loss: 0.009057005860415579]


100%|████████████████████████████████████████████| 50/50 [04:33<00:00, 273.95s/trial, best loss: 0.0074587107085775095]
Beginning fold 7 of 10
100%|██████████████████████████████████████████████| 36/36 [00:52<00:00, 52.55s/trial, best loss: 0.010122535961640922]


100%|█████████████████████████████████████████████| 50/50 [00:55<00:00, 55.05s/trial, best loss: 0.0074587107085775095]
Beginning fold 8 of 10
100%|██████████████████████████████████████████████| 50/50 [00:15<00:00, 15.33s/trial, best loss: 0.006393180607352167]
Beginning fold 9 of 10
100%|█████████████████████████████████████████████████| 3/3 [02:31<00:00, 151.96s/trial, best loss: 0.0431539690996271]


100%|██████████████████████████████████████████████| 50/50 [00:46<00:00, 46.36s/trial, best loss: 0.006393180607352167]
Beginning fold 10 of 10
100%|██████████████████████████████████████████████| 21/21 [01:11<00:00, 71.19s/trial, best loss: 0.009057005860415579]


100%|██████████████████████████████████████████████| 50/50 [00:40<00:00, 40.60s/trial, best loss: 0.009057005860415579]
Retraining model on full dataset
100%|██████████████████████████████████████████████| 39/39 [00:15<00:00, 15.33s/trial, best loss: 0.008633093525179825]


100%|███████████████████████████████████████████████| 50/50 [00:23<00:00, 23.46s/trial, best loss: 0.00767386091127098]


In [32]:
# CLASSIFIER lightGBM 200
unique_id = 'lightGBM_'+ d1 +'_200_balanced' # Unique ID for output file paths
clf = lightgbm_classification(unique_id)  # Classifier
AZKABANML.main_classify(df_200, clf, unique_id, classifypath, preprocessing=any_preprocessing(unique_id))

Beginning fold 1 of 10
100%|███████████████████████████████████████████████████████████████████| 1/1 [05:04<00:00, 304.07s/trial, best loss=?]
Beginning fold 1 of 10
100%|███████████████████████████████████████████████| 50/50 [00:08<00:00,  8.83s/trial, best loss: 0.01006889242183362]
Beginning fold 2 of 10
100%|███████████████████████████████████████████████| 16/16 [00:58<00:00, 58.77s/trial, best loss: 0.01006889242183362]


100%|██████████████████████████████████████████████| 50/50 [00:30<00:00, 30.51s/trial, best loss: 0.006889242183359845]
Beginning fold 3 of 10
100%|██████████████████████████████████████████████| 34/34 [00:32<00:00, 32.33s/trial, best loss: 0.007419183889772141]


100%|██████████████████████████████████████████████| 50/50 [00:27<00:00, 27.84s/trial, best loss: 0.007419183889772141]
Beginning fold 4 of 10
100%|██████████████████████████████████████████████| 50/50 [00:21<00:00, 21.34s/trial, best loss: 0.007419183889772141]
Beginning fold 5 of 10
100%|████████████████████████████████████████████████| 1/1 [00:43<00:00, 43.70s/trial, best loss: 0.022257551669316422]


100%|██████████████████████████████████████████████| 50/50 [00:22<00:00, 22.23s/trial, best loss: 0.006889242183359845]
Beginning fold 6 of 10
100%|██████████████████████████████████████████████| 19/19 [00:17<00:00, 17.39s/trial, best loss: 0.007949125596184436]


100%|██████████████████████████████████████████████| 50/50 [00:59<00:00, 59.48s/trial, best loss: 0.007949125596184436]
Beginning fold 7 of 10
100%|██████████████████████████████████████████████| 37/37 [01:24<00:00, 84.52s/trial, best loss: 0.009009009009009028]


100%|█████████████████████████████████████████████| 50/50 [03:43<00:00, 223.48s/trial, best loss: 0.009009009009009028]
Beginning fold 8 of 10
100%|██████████████████████████████████████████████| 50/50 [00:18<00:00, 18.33s/trial, best loss: 0.005826271186440635]
Beginning fold 9 of 10
100%|████████████████████████████████████████████████| 4/4 [00:44<00:00, 44.07s/trial, best loss: 0.025423728813559365]


100%|█████████████████████████████████████████████| 50/50 [03:12<00:00, 192.78s/trial, best loss: 0.006355932203389814]
Beginning fold 10 of 10
100%|██████████████████████████████████████████████| 22/22 [00:28<00:00, 28.66s/trial, best loss: 0.010593220338983023]


100%|███████████████████████████████████████████████| 50/50 [00:27<00:00, 27.32s/trial, best loss: 0.00794491525423724]
Retraining model on full dataset
100%|██████████████████████████████████████████████| 40/40 [00:19<00:00, 19.77s/trial, best loss: 0.005722460658083017]


100%|██████████████████████████████████████████████| 50/50 [00:33<00:00, 33.25s/trial, best loss: 0.005722460658083017]


In [29]:
# CLASSIFIER SVC 120
unique_id = 'SVC_'+d1+'_120_balanced' # Unique ID for output file paths
clf = svc(unique_id)  # Classifier
AZKABANML.main_classify(df_120, clf, unique_id, classifypath, preprocessing=any_preprocessing(unique_id))

Beginning fold 1 of 10
 95%|██████████████████████████████████████████████████████████████████████▍   | 20/21 [00:00<?, ?trial/s, best loss=?]

job exception: C <= 0



 95%|██████████████████████████████████████████████████████████████████████▍   | 20/21 [00:03<?, ?trial/s, best loss=?]
Beginning fold 1 of 10
100%|██████████████████████████████████████████████| 50/50 [00:04<00:00,  4.41s/trial, best loss: 0.016524520255863484]
Beginning fold 2 of 10
100%|███████████████████████████████████████████████| 16/16 [00:04<00:00,  4.23s/trial, best loss: 0.02238805970149249]


100%|██████████████████████████████████████████████| 50/50 [00:06<00:00,  6.08s/trial, best loss: 0.015991471215351827]
Beginning fold 3 of 10
100%|██████████████████████████████████████████████| 34/34 [00:04<00:00,  4.75s/trial, best loss: 0.019189765458422214]


100%|██████████████████████████████████████████████| 50/50 [01:22<00:00, 82.03s/trial, best loss: 0.009061833688699394]
Beginning fold 4 of 10
100%|██████████████████████████████████████████████| 50/50 [00:18<00:00, 18.57s/trial, best loss: 0.008528784648187626]
Beginning fold 5 of 10
100%|██████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.95s/trial, best loss: 0.0863539445628998]


...

job exception: C <= 0



 96%|███████████████████████████████████████████████████████████████████████▏  | 25/26 [00:04<?, ?trial/s, best loss=?]
Resuming cross-validation from fold 10
Beginning fold 10 of 10
 97%|███████████████████████████████████████████████████████████████████████▋  | 31/32 [00:00<?, ?trial/s, best loss=?]

job exception: C <= 0



 97%|███████████████████████████████████████████████████████████████████████▋  | 31/32 [00:04<?, ?trial/s, best loss=?]
Resuming cross-validation from fold 10
Beginning fold 10 of 10
100%|██████████████████████████████████████████████| 50/50 [00:57<00:00, 57.17s/trial, best loss: 0.013851891315929676]
Retraining model on full dataset
100%|██████████████████████████████████████████████| 16/16 [00:04<00:00,  4.89s/trial, best loss: 0.018705035971222972]


100%|██████████████████████████████████████████████| 50/50 [00:04<00:00,  4.57s/trial, best loss: 0.008633093525179825]


In [30]:
# CLASSIFIER SVC 120
unique_id = 'SVC_'+d1+'_200_balanced' # Unique ID for output file paths
clf = svc(unique_id)  # Classifier
AZKABANML.main_classify(df_200, clf, unique_id, classifypath, preprocessing=any_preprocessing(unique_id))

Beginning fold 1 of 10
100%|██████████████████████████████████████████████| 50/50 [00:11<00:00, 11.91s/trial, best loss: 0.014838367779544281]
Beginning fold 2 of 10
100%|██████████████████████████████████████████████| 17/17 [00:38<00:00, 38.54s/trial, best loss: 0.018547959724430352]


100%|██████████████████████████████████████████████| 50/50 [00:46<00:00, 46.14s/trial, best loss: 0.018018018018018056]
Beginning fold 3 of 10
100%|███████████████████████████████████████████████| 35/35 [00:04<00:00,  4.69s/trial, best loss: 0.02119766825649183]


100%|███████████████████████████████████████████████| 50/50 [00:05<00:00,  5.26s/trial, best loss: 0.02119766825649183]
Beginning fold 4 of 10
100%|██████████████████████████████████████████████| 50/50 [00:04<00:00,  4.19s/trial, best loss: 0.018018018018018056]
Beginning fold 5 of 10
100%|█████████████████████████████████████████████████| 2/2 [00:05<00:00,  5.46s/trial, best loss: 0.14361420243773182]


100%|██████████████████████████████████████████████| 50/50 [00:04<00:00,  4.09s/trial, best loss: 0.028086910439851565]
Beginning fold 6 of 10
100%|██████████████████████████████████████████████| 20/20 [00:04<00:00,  4.53s/trial, best loss: 0.027027027027026973]


100%|██████████████████████████████████████████████| 50/50 [00:06<00:00,  6.49s/trial, best loss: 0.026497085320614677]
Beginning fold 7 of 10
100%|██████████████████████████████████████████████| 38/38 [00:14<00:00, 14.24s/trial, best loss: 0.014308426073131986]


...

job exception: C <= 0



 98%|████████████████████████████████████████████████████████████████████████▎ | 42/43 [00:03<?, ?trial/s, best loss=?]
Resuming cross-validation from fold 7
Beginning fold 7 of 10
100%|███████████████████████████████████████████████| 50/50 [00:04<00:00,  4.07s/trial, best loss: 0.03762586115527289]
Beginning fold 8 of 10
100%|███████████████████████████████████████████████| 16/16 [00:05<00:00,  5.14s/trial, best loss: 0.05455508474576276]


100%|██████████████████████████████████████████████| 50/50 [00:04<00:00,  4.46s/trial, best loss: 0.018538135593220373]
Beginning fold 9 of 10
100%|███████████████████████████████████████████████| 34/34 [00:05<00:00,  5.13s/trial, best loss: 0.01906779661016944]


100%|███████████████████████████████████████████████| 50/50 [00:04<00:00,  4.04s/trial, best loss: 0.01906779661016944]
Beginning fold 10 of 10
100%|███████████████████████████████████████████████████████████████████| 1/1 [05:03<00:00, 303.56s/trial, best loss=?]
Resuming cross-validation from fold 10
Beginning fold 10 of 10
100%|███████████████████████████████████████████████| 50/50 [00:06<00:00,  6.41s/trial, best loss: 0.01959745762711862]
Retraining model on full dataset


100%|██████████████████████████████████████████████| 50/50 [00:21<00:00, 21.53s/trial, best loss: 0.014306151645207432]


## Linear
Balanced + PP

In [33]:
# CLASSIFIER kNN 120
unique_id = 'kNN_'+ d1 +'_120_sigbs' # Unique ID for output file paths
clf = k_neighbors_classifier(unique_id)  # Classifier
AZKABANML.main_classify(df_120_sigbs, clf, unique_id, classifypath, preprocessing=any_preprocessing(unique_id))

Beginning fold 1 of 10
100%|██████████████████████████████████████████████| 50/50 [00:04<00:00,  4.00s/trial, best loss: 0.009594882729211052]
Beginning fold 2 of 10
100%|██████████████████████████████████████████████| 17/17 [00:04<00:00,  4.29s/trial, best loss: 0.009061833688699394]


100%|██████████████████████████████████████████████| 50/50 [00:05<00:00,  5.82s/trial, best loss: 0.007995735607675858]
Beginning fold 3 of 10
100%|██████████████████████████████████████████████| 35/35 [00:04<00:00,  4.49s/trial, best loss: 0.009594882729211052]


100%|██████████████████████████████████████████████| 50/50 [00:04<00:00,  4.08s/trial, best loss: 0.009594882729211052]
Beginning fold 4 of 10
100%|██████████████████████████████████████████████| 50/50 [00:04<00:00,  4.37s/trial, best loss: 0.006929637526652432]
Beginning fold 5 of 10
100%|████████████████████████████████████████████████| 2/2 [00:05<00:00,  5.36s/trial, best loss: 0.029317697228145034]


100%|███████████████████████████████████████████████| 50/50 [00:04<00:00,  4.03s/trial, best loss: 0.01012793176972282]
Beginning fold 6 of 10
100%|██████████████████████████████████████████████| 20/20 [00:04<00:00,  4.12s/trial, best loss: 0.008524240809802852]


100%|██████████████████████████████████████████████| 50/50 [00:06<00:00,  6.05s/trial, best loss: 0.008524240809802852]
Beginning fold 7 of 10
100%|██████████████████████████████████████████████| 38/38 [00:05<00:00,  5.17s/trial, best loss: 0.008524240809802852]


100%|██████████████████████████████████████████████| 50/50 [00:05<00:00,  5.02s/trial, best loss: 0.008524240809802852]
Beginning fold 8 of 10
100%|██████████████████████████████████████████████| 50/50 [00:04<00:00,  4.06s/trial, best loss: 0.005327650506126824]
Beginning fold 9 of 10
100%|████████████████████████████████████████████████| 5/5 [00:04<00:00,  4.13s/trial, best loss: 0.008524240809802852]


100%|█████████████████████████████████████████████| 50/50 [00:06<00:00,  6.10s/trial, best loss: 0.0074587107085775095]
Beginning fold 10 of 10
100%|██████████████████████████████████████████████| 23/23 [00:06<00:00,  6.02s/trial, best loss: 0.007991475759190236]


100%|██████████████████████████████████████████████| 50/50 [00:04<00:00,  4.10s/trial, best loss: 0.007991475759190236]
Retraining model on full dataset
100%|██████████████████████████████████████████████| 41/41 [00:04<00:00,  4.01s/trial, best loss: 0.005275779376498813]


100%|██████████████████████████████████████████████| 50/50 [00:03<00:00,  3.99s/trial, best loss: 0.005275779376498813]


In [34]:
# CLASSIFIER kNN 200
unique_id = 'kNN_'+ d1 +'_200_sigbs' # Unique ID for output file paths
clf = k_neighbors_classifier(unique_id)  # Classifier

AZKABANML.main_classify(df_200_sigbs, clf, unique_id, classifypath, preprocessing=any_preprocessing(unique_id))

Beginning fold 1 of 10
100%|██████████████████████████████████████████████| 50/50 [00:04<00:00,  4.55s/trial, best loss: 0.009538950715421324]
Beginning fold 2 of 10
100%|██████████████████████████████████████████████| 17/17 [00:07<00:00,  7.00s/trial, best loss: 0.015368309485956577]


100%|███████████████████████████████████████████████| 50/50 [00:06<00:00,  6.21s/trial, best loss: 0.01377848436671969]
Beginning fold 3 of 10
100%|██████████████████████████████████████████████| 35/35 [00:04<00:00,  4.98s/trial, best loss: 0.011658717541070507]


100%|██████████████████████████████████████████████| 50/50 [00:06<00:00,  6.85s/trial, best loss: 0.011658717541070507]
Beginning fold 4 of 10
100%|██████████████████████████████████████████████| 50/50 [00:06<00:00,  6.40s/trial, best loss: 0.008479067302596732]
Beginning fold 5 of 10
100%|█████████████████████████████████████████████████| 2/2 [00:04<00:00,  4.74s/trial, best loss: 0.01642819289878117]


100%|██████████████████████████████████████████████| 50/50 [00:04<00:00,  4.71s/trial, best loss: 0.012188659247482803]
Beginning fold 6 of 10
100%|███████████████████████████████████████████████| 20/20 [00:07<00:00,  7.60s/trial, best loss: 0.01377848436671969]


100%|██████████████████████████████████████████████| 50/50 [00:06<00:00,  6.65s/trial, best loss: 0.011128775834658211]
Beginning fold 7 of 10
100%|██████████████████████████████████████████████| 38/38 [00:05<00:00,  5.95s/trial, best loss: 0.015898251192368873]


100%|██████████████████████████████████████████████| 50/50 [00:05<00:00,  5.13s/trial, best loss: 0.015368309485956577]
Beginning fold 8 of 10
100%|██████████████████████████████████████████████| 50/50 [00:04<00:00,  4.46s/trial, best loss: 0.012711864406779627]
Beginning fold 9 of 10
100%|█████████████████████████████████████████████████| 5/5 [00:04<00:00,  4.50s/trial, best loss: 0.01641949152542377]


100%|██████████████████████████████████████████████| 50/50 [00:06<00:00,  6.67s/trial, best loss: 0.010063559322033844]
Beginning fold 10 of 10
100%|██████████████████████████████████████████████| 23/23 [00:04<00:00,  4.50s/trial, best loss: 0.012711864406779627]


100%|███████████████████████████████████████████████| 50/50 [00:05<00:00,  5.67s/trial, best loss: 0.01165254237288138]
Retraining model on full dataset
100%|██████████████████████████████████████████████| 41/41 [00:04<00:00,  4.51s/trial, best loss: 0.010968049594659024]


100%|██████████████████████████████████████████████| 50/50 [00:06<00:00,  6.33s/trial, best loss: 0.010491177873152124]


In [35]:
# CLASSIFIER lightGBM 120
unique_id = 'lightGBM_'+ d1 +'_120_sigbs' # Unique ID for output file paths
clf = lightgbm_classification(unique_id)  # Classifier
AZKABANML.main_classify(df_120_sigbs, clf, unique_id, classifypath, preprocessing=any_preprocessing(unique_id))

Beginning fold 1 of 10
100%|██████████████████████████████████████████████| 50/50 [00:37<00:00, 37.26s/trial, best loss: 0.006929637526652432]
Beginning fold 2 of 10
100%|██████████████████████████████████████████████| 17/17 [00:53<00:00, 53.67s/trial, best loss: 0.008528784648187626]


100%|██████████████████████████████████████████████| 50/50 [00:36<00:00, 36.44s/trial, best loss: 0.005863539445629007]
Beginning fold 3 of 10
100%|██████████████████████████████████████████████| 35/35 [00:17<00:00, 17.71s/trial, best loss: 0.005330490405117239]


100%|██████████████████████████████████████████████| 50/50 [00:15<00:00, 15.52s/trial, best loss: 0.005330490405117239]
Beginning fold 4 of 10
100%|█████████████████████████████████████████████| 50/50 [03:04<00:00, 184.51s/trial, best loss: 0.006929637526652432]
Beginning fold 5 of 10
100%|████████████████████████████████████████████████| 2/2 [00:08<00:00,  8.09s/trial, best loss: 0.021855010660980834]


100%|██████████████████████████████████████████████| 50/50 [00:20<00:00, 20.92s/trial, best loss: 0.006396588486140775]
Beginning fold 6 of 10
100%|██████████████████████████████████████████████| 20/20 [00:15<00:00, 15.44s/trial, best loss: 0.010122535961640922]


100%|██████████████████████████████████████████████| 50/50 [00:18<00:00, 18.46s/trial, best loss: 0.009589770911028195]
Beginning fold 7 of 10
100%|██████████████████████████████████████████████| 38/38 [00:38<00:00, 38.75s/trial, best loss: 0.009589770911028195]


100%|██████████████████████████████████████████████| 50/50 [00:30<00:00, 30.20s/trial, best loss: 0.009589770911028195]
Beginning fold 8 of 10
100%|██████████████████████████████████████████████| 50/50 [00:09<00:00,  9.88s/trial, best loss: 0.005327650506126824]
Beginning fold 9 of 10
100%|████████████████████████████████████████████████| 5/5 [00:22<00:00, 22.34s/trial, best loss: 0.010655301012253648]


100%|███████████████████████████████████████████████| 50/50 [00:35<00:00, 35.27s/trial, best loss: 0.00586041555673944]
Beginning fold 10 of 10
100%|██████████████████████████████████████████████| 23/23 [00:07<00:00,  7.10s/trial, best loss: 0.006393180607352167]


100%|███████████████████████████████████████████████| 50/50 [00:22<00:00, 22.10s/trial, best loss: 0.00586041555673944]
Retraining model on full dataset
100%|██████████████████████████████████████████████| 41/41 [00:13<00:00, 13.90s/trial, best loss: 0.005275779376498813]


100%|█████████████████████████████████████████████| 50/50 [01:51<00:00, 111.22s/trial, best loss: 0.005275779376498813]


In [36]:
# CLASSIFIER lightGBM 200
unique_id = 'lightGBM_'+ d1 +'_200_sigbs' # Unique ID for output file paths
clf = lightgbm_classification(unique_id)  # Classifier
AZKABANML.main_classify(df_200_sigbs, clf, unique_id, classifypath, preprocessing=any_preprocessing(unique_id))

Beginning fold 1 of 10
100%|██████████████████████████████████████████████| 50/50 [01:11<00:00, 71.87s/trial, best loss: 0.005299417064122958]
Beginning fold 2 of 10
100%|██████████████████████████████████████████████| 17/17 [01:56<00:00, 116.82s/trial, best loss: 0.01006889242183362]


100%|██████████████████████████████████████████████| 50/50 [00:10<00:00, 10.73s/trial, best loss: 0.008479067302596732]
Beginning fold 3 of 10
100%|██████████████████████████████████████████████| 35/35 [00:31<00:00, 31.79s/trial, best loss: 0.005829358770535253]


100%|█████████████████████████████████████████████| 50/50 [01:47<00:00, 107.92s/trial, best loss: 0.005829358770535253]
Beginning fold 4 of 10
100%|█████████████████████████████████████████████| 50/50 [05:04<00:00, 304.38s/trial, best loss: 0.005299417064122958]
Beginning fold 5 of 10
100%|███████████████████████████████████████████████| 2/2 [04:38<00:00, 278.07s/trial, best loss: 0.022787493375728718]


100%|██████████████████████████████████████████████| 50/50 [00:18<00:00, 18.12s/trial, best loss: 0.008479067302596732]
Beginning fold 6 of 10
100%|██████████████████████████████████████████████| 20/20 [00:17<00:00, 17.77s/trial, best loss: 0.011658717541070507]


100%|██████████████████████████████████████████████| 50/50 [00:29<00:00, 29.16s/trial, best loss: 0.009009009009009028]
Beginning fold 7 of 10
100%|██████████████████████████████████████████████| 38/38 [00:48<00:00, 48.27s/trial, best loss: 0.009009009009009028]


100%|██████████████████████████████████████████████| 50/50 [01:37<00:00, 97.58s/trial, best loss: 0.009009009009009028]
Beginning fold 8 of 10
100%|███████████████████████████████████████████████| 50/50 [00:08<00:00,  8.85s/trial, best loss: 0.00794491525423724]
Beginning fold 9 of 10
100%|████████████████████████████████████████████████| 5/5 [00:33<00:00, 33.18s/trial, best loss: 0.028601694915254217]


100%|███████████████████████████████████████████████| 50/50 [01:25<00:00, 85.50s/trial, best loss: 0.00794491525423724]
Beginning fold 10 of 10
100%|███████████████████████████████████████████████| 23/23 [00:37<00:00, 37.80s/trial, best loss: 0.00794491525423724]


100%|██████████████████████████████████████████████| 50/50 [01:06<00:00, 66.89s/trial, best loss: 0.007415254237288171]
Retraining model on full dataset
100%|█████████████████████████████████████████████| 41/41 [00:20<00:00, 20.12s/trial, best loss: 0.0066762041010968165]


100%|█████████████████████████████████████████████| 50/50 [00:18<00:00, 18.33s/trial, best loss: 0.0066762041010968165]


In [37]:
# CLASSIFIER SVC 120
unique_id = 'SVC_'+ d1 +'_120_sigbs' # Unique ID for output file paths
clf = svc(unique_id)  # Classifier
AZKABANML.main_classify(df_120_sigbs, clf, unique_id, classifypath, preprocessing=any_preprocessing(unique_id))

Beginning fold 1 of 10
100%|███████████████████████████████████████████████| 50/50 [00:05<00:00,  5.15s/trial, best loss: 0.04477611940298509]
Beginning fold 2 of 10
100%|███████████████████████████████████████████████| 17/17 [00:04<00:00,  4.09s/trial, best loss: 0.02505330490405122]


100%|██████████████████████████████████████████████| 50/50 [00:07<00:00,  7.04s/trial, best loss: 0.012260127931769671]
Beginning fold 3 of 10
100%|███████████████████████████████████████████████| 35/35 [00:04<00:00,  4.68s/trial, best loss: 0.01279317697228144]


100%|███████████████████████████████████████████████| 50/50 [00:04<00:00,  4.55s/trial, best loss: 0.01279317697228144]
Beginning fold 4 of 10
100%|███████████████████████████████████████████████| 50/50 [00:05<00:00,  5.15s/trial, best loss: 0.04637526652452029]
Beginning fold 5 of 10
100%|██████████████████████████████████████████████████| 2/2 [00:05<00:00,  5.63s/trial, best loss: 0.6407249466950959]


100%|██████████████████████████████████████████████| 50/50 [00:04<00:00,  4.97s/trial, best loss: 0.013326226012793208]
Beginning fold 6 of 10
100%|██████████████████████████████████████████████| 20/20 [00:05<00:00,  5.01s/trial, best loss: 0.017048481619605704]


100%|██████████████████████████████████████████████| 50/50 [00:04<00:00,  4.91s/trial, best loss: 0.012253596164091607]
Beginning fold 7 of 10
100%|███████████████████████████████████████████████| 38/38 [00:16<00:00, 16.01s/trial, best loss: 0.04741608950452847]


100%|███████████████████████████████████████████████| 50/50 [00:05<00:00,  5.28s/trial, best loss: 0.04688332445391585]
Beginning fold 8 of 10
100%|██████████████████████████████████████████████| 50/50 [00:04<00:00,  4.52s/trial, best loss: 0.013851891315929676]
Beginning fold 9 of 10
100%|█████████████████████████████████████████████████| 5/5 [00:04<00:00,  4.33s/trial, best loss: 0.07618540223761316]


100%|███████████████████████████████████████████████| 50/50 [00:06<00:00,  6.53s/trial, best loss: 0.05007991475759188]
Beginning fold 10 of 10
100%|██████████████████████████████████████████████| 23/23 [00:04<00:00,  4.01s/trial, best loss: 0.021310602024507186]


100%|██████████████████████████████████████████████| 50/50 [00:06<00:00,  6.00s/trial, best loss: 0.010122535961640922]
Retraining model on full dataset
100%|███████████████████████████████████████████████| 41/41 [00:07<00:00,  7.61s/trial, best loss: 0.04652278177458036]


100%|███████████████████████████████████████████████| 50/50 [00:06<00:00,  6.16s/trial, best loss: 0.04652278177458036]


In [38]:
# CLASSIFIER SVC 120
unique_id = 'SVC_'+ d1 +'_200_sigbs' # Unique ID for output file paths
clf = svc(unique_id)  # Classifier
AZKABANML.main_classify(df_200_sigbs, clf, unique_id, classifypath, preprocessing=any_preprocessing(unique_id))

Beginning fold 1 of 10
100%|██████████████████████████████████████████████| 50/50 [00:05<00:00,  5.74s/trial, best loss: 0.025437201907790197]
Beginning fold 2 of 10
100%|███████████████████████████████████████████████| 17/17 [00:04<00:00,  4.34s/trial, best loss: 0.12347641759406469]


 96%|███████████████████████████████████████████████████████████████████████   | 24/25 [00:00<?, ?trial/s, best loss=?]

job exception: C <= 0



 96%|███████████████████████████████████████████████████████████████████████   | 24/25 [00:03<?, ?trial/s, best loss=?]
Resuming cross-validation from fold 2
Beginning fold 2 of 10
100%|███████████████████████████████████████████████| 50/50 [00:04<00:00,  4.55s/trial, best loss: 0.04981452040275569]
Beginning fold 3 of 10
100%|███████████████████████████████████████████████| 16/16 [00:06<00:00,  6.38s/trial, best loss: 0.13831478537360886]


100%|███████████████████████████████████████████████| 50/50 [00:54<00:00, 54.76s/trial, best loss: 0.04080551139374666]
Beginning fold 4 of 10
100%|██████████████████████████████████████████████| 34/34 [01:12<00:00, 72.78s/trial, best loss: 0.032856385797562226]


...

100%|████████████████████████████████████████████████| 50/50 [00:05<00:00,  5.15s/trial, best loss: 0.1229464758876524]
Beginning fold 7 of 10
100%|████████████████████████████████████████████████| 19/19 [00:05<00:00,  5.40s/trial, best loss: 0.1324854266030736]


100%|███████████████████████████████████████████████| 50/50 [00:04<00:00,  4.75s/trial, best loss: 0.11075781664016959]
Beginning fold 8 of 10
100%|███████████████████████████████████████████████| 37/37 [00:05<00:00,  5.25s/trial, best loss: 0.13559322033898302]


100%|███████████████████████████████████████████████| 50/50 [00:05<00:00,  5.15s/trial, best loss: 0.13559322033898302]
Beginning fold 9 of 10
100%|███████████████████████████████████████████████| 50/50 [00:05<00:00,  5.52s/trial, best loss: 0.10858050847457623]
Beginning fold 10 of 10
100%|█████████████████████████████████████████████████| 4/4 [00:06<00:00,  6.41s/trial, best loss: 0.13718220338983056]


100%|██████████████████████████████████████████████| 50/50 [00:05<00:00,  5.38s/trial, best loss: 0.056144067796610186]
Retraining model on full dataset
100%|███████████████████████████████████████████████| 22/22 [00:04<00:00,  4.65s/trial, best loss: 0.14020028612303292]


100%|████████████████████████████████████████████████| 50/50 [00:05<00:00,  5.15s/trial, best loss: 0.0848831664282308]
